<a href="https://colab.research.google.com/github/artemkush1/ML_predict_CS-GO_matches/blob/master/data_collect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test proxies

In [ ]:
from sys import argv
import urllib3
from os import system as terminal
import requests
from requests.auth import HTTPProxyAuth
from colorama import Fore,Style

URL = "http://www.hltv.org"
auth = HTTPProxyAuth('jfhh9d', '95K7TB')
CMD_CLEAR_TERM = "clear"
TIMEOUT = (3.05,27)

def check_proxy(proxy):
    '''
        Function for check proxy return ERROR
        if proxy is Bad else
        Function return None
    '''
    try:
        session = requests.Session()
        session.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
        session.max_redirects = 300
        proxy = proxy.split('\n',1)[0]
        print(Fore.LIGHTYELLOW_EX + 'Checking ' + proxy)
        session.get(URL, proxies={'http':'http://' + proxy}, timeout=TIMEOUT,allow_redirects=True, auth=auth)
    except requests.exceptions.ConnectionError as e:
        print(Fore.LIGHTRED_EX + 'Error!')
        return e
    except requests.exceptions.ConnectTimeout as e:
        print(Fore.LIGHTRED_EX + 'Error,Timeout!')
        return e
    except requests.exceptions.HTTPError as e:
        print(Fore.LIGHTRED_EX + 'HTTP ERROR!')
        return e
    except requests.exceptions.Timeout as e:
        print(Fore.LIGHTRED_EX + 'Error! Connection Timeout!')
        return e
    except urllib3.exceptions.ProxySchemeUnknown as e:
        print(Fore.LIGHTRED_EX + 'ERROR unkown Proxy Scheme!')
        return e
    except requests.exceptions.TooManyRedirects as e:
        print(Fore.LIGHTRED_EX + 'ERROR! Too many redirects!')
        return e

"""with open('proxies.txt', 'r') as f:
    data_proxy = f.read().split()
if data_proxy:
    proxies = data_proxy
else:"""
proxies = ['213.166.73.89:9856']
goods = 0
terminal(CMD_CLEAR_TERM)
print(Fore.LIGHTCYAN_EX + '===========================================')
for proxy in proxies:
    try:
        if check_proxy(proxy):
            print(Fore.LIGHTRED_EX + 'Bad proxy ' + proxy)
        else:
            print(Fore.LIGHTGREEN_EX + 'Good proxy ' + proxy)
            with open('good.txt','a') as file_with_goods:
                file_with_goods.write(proxy + '\n')
            goods += 1
        print(Fore.LIGHTCYAN_EX + '=================================================')
    except KeyboardInterrupt:
        print(Fore.LIGHTGREEN_EX + '\nExit.')
        exit()

print(Fore.LIGHTGREEN_EX + 'Total ' + str(goods) + ' good proxies found')
print(Fore.LIGHTRED_EX + 'And ' + str(len(proxies) - goods) + ' is bad')
print()

In [ ]:
with open('good.txt', 'r') as f:
    data = f.read().split()
data

# Global params

In [4]:
import requests
import urllib
import datetime
from bs4 import BeautifulSoup
from re import sub
import re
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
from datetime import datetime
import time
from requests.auth import HTTPProxyAuth
import random

PREFIX = 'https://www.hltv.org'
proxyDict = {'http': 'http://213.166.73.89:9856'}
proxyAuth = HTTPProxyAuth('jfhh9d', '95K7TB')

## Settings for parser

In [5]:
TIMESLEEP = 0.01
def get_parsed_page(url, proxy_=None, auth_=None):
    # This fixes a blocked by cloudflare error i've encountered
    time.sleep(TIMESLEEP)
    got_req = requests.get(url, proxies=proxy_, auth=auth_).text
    return BeautifulSoup(got_req, "lxml")

## Ids each map:

In [489]:
id_maps = {
'Dust2': 31,
'Inferno': 33,
'Mirage': 32,
'Nuke': 34,
'Overpass': 40,
'Train': 35,
'Vertigo': 46,
'Cache': 29,
'Cobblestone': 39,
}
prefix_map_stat = 'http://www.hltv.org/stats/teams/map/'
prefix_player_stat = 'http://www.hltv.org/stats/players/'

### Examples

In [150]:
# match_url: http://www.hltv.org/matches/2341715/vitality-vs-dignitas-blast-premier-spring-2020-europe-showdown
# map_stat_url for a team: http://www.hltv.org/stats/teams/map/35/4991/fnatic
# event_url: http://www.hltv.org/events/5205/blast-premier-spring-2020-europe-showdown
# map_stat from match: https://www.hltv.org/stats/matches/mapstatsid/103700/fnatic-vs-vitality
# plyer_stat: http://www.hltv.org/stats/players/3055/flusha

# Parse matches, transform data to pandas frame and save csv

In [490]:
import urllib3
from os import system as terminal
import requests
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup


PAGES = [i for i in range(40)]
TIMESLEEP = random.uniform(0.25, 0.3)

# download all urls matches
def get_matches_url(filename=None, pages_with_results=[0]):
    all_matches = []
    for i in pages_with_results:
        if i == 0:
            results = get_parsed_page('http://www.hltv.org/results', proxyDict, proxyAuth)
        else:
            results = get_parsed_page(f'http://www.hltv.org/results?offset={i}00', proxyDict, proxyAuth)
        # list of matches
        all_matches += [url_.find('a', {"class": "a-reset"})['href'] for url_ in
                        results.find('div',
                                     {"class": "results-all", 'data-zonedgrouping-group-classes': "results-sublist"}).
                        find_all('div', {"class": "result-con"})]
    all_matches = np.array(all_matches)
    if filename is None:
        return all_matches
    else:
        pd.DataFrame(all_matches, columns=['match_url']).to_csv(f'{filename}.csv')



get_matches_url(filename='df_matches', pages_with_results=PAGES)
#df = pd.DataFrame(data, columns=['match_url'])
#df.to_csv('df_matches.csv')

# Add links in df and parse data

In [ ]:
df = pd.read_csv('df_matches.csv').set_index('Unnamed: 0')
df

In [ ]:
TIMESLEEP = 0.1


class MatchPageParams:
    def __init__(self, df, start_index=None, finish_index=None):
        self.df = df
        self.MATCH_PAGE = None
        self.MATCH_ID = None
        self.match = None
        self.start_index = start_index
        self.finish_index = finish_index

    def add_all_params(self):
        time_start = time.time()
        for match in self.df['match_url'][self.start_index:self.finish_index]:
            self.MATCH_ID = list(self.df['match_url']).index(match)
            print(PREFIX + match)
            print('match id -', self.MATCH_ID, '; time spent -', time.time() - time_start)
            self.match = match
            # self.MATCH_PAGE = get_parsed_page(PREFIX + match, proxyDict, proxyAuth)
            """self.add_url_teams()
            self.add_url_event()
            self.add_10_urls_players()
            self.add_total_maps()
            self.add_maps()
            self.add_date()
            self.add_scores()
            self.add_history()
            self.add_ranks()"""
            #self.add_5last_urls_opponents()
            self.add_5last_scores_and_types()
            # Лучше просто парсить 15 последних карт
            self.MATCH_PAGE = None
    
    def parse_page(self):
        if self.MATCH_PAGE is None:
            self.MATCH_PAGE = get_parsed_page(PREFIX + self.match, proxyDict, proxyAuth)

    def _get_score_(self):
        self.parse_page()
        score = self.MATCH_PAGE.find('div', {'class': 'match-page'}).\
                find('div', {'class': 'teamsBox'}).\
                find_all('div', {'class': 'team'})
        for i in range(len(score)):
            score[i] = score[i].find('div', {'class': f'team{i + 1}-gradient'}).\
                                find_all('div')[-1].text
            self.df[f'score{i + 1}'][self.MATCH_ID] = score[i]

    def add_scores(self):
        # add columns
        if 'score1' not in self.df.columns:
            self.df['score1'] = ""
        if 'score2' not in self.df.columns:
            self.df['score2'] = ""
        if self.df['score1'][self.MATCH_ID] == "" or self.df['score2'][self.MATCH_ID] == "":
            self._get_score_()

    def _get_total_maps_(self):
        self.parse_page()
        try:
            table = self.MATCH_PAGE.find('div', {'class': 'flexbox-column'}).\
                    find_all('div', {'class': 'mapholder'})
            self.df['total_maps'][self.MATCH_ID] = len(table)
            self.df['maps_played'][self.MATCH_ID] = 0
            for i in table:
                played = i.find('div', {'class': 'results'}).find('div', {'class': 'results-center'})
                if played.text:
                    self.df['maps_played'][self.MATCH_ID] += 1
        except:
            self.df['total_maps'][self.MATCH_ID] = 0
            self.df['maps_played'][self.MATCH_ID] = 0
            

    def add_total_maps(self):
        # add columns
        if 'total_maps' not in self.df:
            self.df['total_maps'] = ""
        if 'maps_played' not in self.df:
            self.df['maps_played'] = ""
        if self.df['total_maps'][self.MATCH_ID] == "" or self.df['maps_played'][self.MATCH_ID] == "":
            self._get_total_maps_()

    def _get_ranks_(self):
        self.parse_page()
        ranks = self.MATCH_PAGE.find('div', {'class': 'lineups'}).\
                find_all('div', {'class': 'box-headline'})
        for i in range(len(ranks)):
            ranks[i] = ranks[i].find('div', {'class': 'teamRanking'}).text
            self.df[f'rank{i + 1}'][self.MATCH_ID] = ranks[i]
            
    def add_ranks(self):
        # add columns
        if 'rank1' not in self.df:
            self.df['rank1'] = ""
        if 'rank2' not in self.df:
            self.df['rank2'] = ""
        if self.df['rank1'][self.MATCH_ID] == "" or self.df['rank2'][self.MATCH_ID] == "":
            self._get_ranks_()
    
    def _get_date_(self):
        self.parse_page()
        self.df['date'][self.MATCH_ID] = datetime.utcfromtimestamp(int(
                self.MATCH_PAGE.find('div', {'class': 'timeAndEvent'}).
                find('div', {'class': 'time'})['data-unix'][:-3])).strftime('%Y-%m-%d')
    
    def add_date(self):
        if 'date' not in self.df:
            self.df['date'] = ""
        if self.df['date'][self.MATCH_ID] == "":
            self._get_date_()

    def _get_history_(self):
        self.parse_page()
        h2h = self.MATCH_PAGE.find('div', {'class': 'head-to-head'}).\
                find_all('div', {'class': 'flexbox-column'})
        self.df['h2h_wins1'][self.MATCH_ID] = h2h[0].find('div', {'class': 'bold'}).text
        self.df['h2h_wins2'][self.MATCH_ID] = h2h[-1].find('div', {'class': 'bold'}).text

    def add_history(self):
        if 'h2h_wins1' not in self.df:
            self.df['h2h_wins1'] = ""
        if 'h2h_wins2' not in self.df:
            self.df['h2h_wins2'] = ""
        if self.df['h2h_wins1'][self.MATCH_ID] == "" or self.df['h2h_wins2'][self.MATCH_ID] == "":
            self._get_history_()

    def _get_5last_score_type_(self, team_id, match_id):
        self.parse_page()
        try:
            tmp = self.MATCH_PAGE.find('div', {'class': 'past-matches'}).\
                    find_all('div', {'class': 'half-width'})[team_id].find('table', {'class': 'matches'}).\
                    find_all('tr', {'class': 'table'})[match_id]
            total = tmp.find('a', {'data-link-tracking-page': 'Matchpage'}).text
            self.df[f'5last_match{match_id + 1}_total_maps{team_id + 1}'][self.MATCH_ID] = total
            score = tmp.find('td', {'class': 'result'}).text.split(' - ')
            if int(score[0]) > int(score[1]):
                score = [1, 0]
            else:
                score = [0, 1]
            self.df[f'5last_match{match_id + 1}_score{team_id + 1}'][self.MATCH_ID] = score[0]
            self.df[f'5last_match{match_id + 1}_opponent_score{team_id + 1}'][self.MATCH_ID] = score[1]
        except:
            self.df[f'5last_match{match_id + 1}_total_maps{team_id + 1}'][self.MATCH_ID] = 0
            self.df[f'5last_match{match_id + 1}_score{team_id + 1}'][self.MATCH_ID] = 0
            self.df[f'5last_match{match_id + 1}_opponent_score{team_id + 1}'][self.MATCH_ID] = 0
        
    
    def add_5last_scores_and_types(self):
        for team_id in range(2):
            for match_id in range(5):
                if f'5last_match{match_id + 1}_total_maps{team_id + 1}' not in self.df:
                    self.df[f'5last_match{match_id + 1}_total_maps{team_id + 1}'] = ""
                if f'5last_match{match_id + 1}_score{team_id + 1}' not in self.df:
                    self.df[f'5last_match{match_id + 1}_score{team_id + 1}'] = ""
                if f'5last_match{match_id + 1}_opponent_score{team_id + 1}' not in self.df:
                    self.df[f'5last_match{match_id + 1}_opponent_score{team_id + 1}'] = ""
                if self.df[f'5last_match{match_id + 1}_total_maps{team_id + 1}'][self.MATCH_ID] == "" or\
                    self.df[f'5last_match{match_id + 1}_score{team_id + 1}'][self.MATCH_ID] == "" or\
                    self.df[f'5last_match{match_id + 1}_opponent_score{team_id + 1}'][self.MATCH_ID] == "":
                    self._get_5last_score_type_(team_id, match_id)

    def _get_5last_opponents_(self, team_id):
        self.parse_page()
        arr = []
        tmp = self.MATCH_PAGE.find('div', {'class': 'past-matches'}).\
                find_all('div', {'class': 'half-width'})[team_id].find('table', {'class': 'matches'}).\
                find_all('tr', {'class': 'table'})
        for i in tmp:
            arr += [i.find('td', {'class': 'opponent'}).find('a')['href']]
        self.df[f'opponents_url_history_team{team_id + 1}'][self.MATCH_ID] = arr
        
    
    def add_5last_urls_opponents(self):
        for team_id in range(2):
            if f'opponents_url_history_team{team_id + 1}' not in self.df:
                self.df[f'opponents_url_history_team{team_id + 1}'] = ""
            if self.df[f'opponents_url_history_team{team_id + 1}'][self.MATCH_ID] == "":
                self._get_5last_opponents_(team_id)

    def _get_players_url__(self, team_id):
        self.parse_page()
        arr = []
        tmp = self.MATCH_PAGE.find('div', {'class': 'lineups'}).\
                find_all('div', {'class': 'lineup'})[team_id].find('div', {'class': 'players'}).\
                find_all('td', {'class': 'player'})
        for i in tmp:
            try:
                arr += [i.find('a')['href']]
            except:
                arr += [""]
        self.df[f'players_url_{team_id + 1}'][self.MATCH_ID] = arr[:5]
        #print(arr)

    def add_10_urls_players(self):
        for team_id in range(2):
            if f'players_url_{team_id + 1}' not in self.df:
                self.df[f'players_url_{team_id + 1}'] = ""
            if self.df[f'players_url_{team_id + 1}'][self.MATCH_ID] == "":
                self._get_players_url__(team_id)

    def _get_event_url_(self):
        self.parse_page()
        self.df['event_url'][self.MATCH_ID] = self.MATCH_PAGE.find('div', {'class': 'timeAndEvent'}).\
                find('div', {'class': 'event'}).find('a')['href']
    
    def add_url_event(self):
        if 'event_url' not in self.df:
            self.df['event_url'] = ""
        if self.df['event_url'][self.MATCH_ID] == "":
            self._get_event_url_()
    
    def _get_maps_(self):
        self.parse_page()
        arr = []
        map_name = []
        score_maps1 = []
        score_maps2 = []
        picks = []
        try:
            tmp = self.MATCH_PAGE.find('div', {'class': 'flexbox-column'}).\
                    find_all('div', {'class': 'mapholder'})
            for i in tmp:
                played = i.find('div', {'class': 'results'}).find('div', {'class': 'results-center'})
                if played.text:
                    arr += [played.find('a')['href']]
                    map_name += [i.find('div', {'class': 'map-name-holder'}).find('div', {'class': 'mapname'}).text]
                    table = i.find('div', {'class': 'results-left'})
                    if self.df['total_maps'][self.MATCH_ID] > 1:
                        if table['class'][-1] == 'pick':
                            picks += [1]
                    score_maps1 += [table.find('div', {'class': 'results-team-score'}).text]
                    table = i.find('span', {'class': 'results-right'})
                    if self.df['total_maps'][self.MATCH_ID] > 1:
                        if table['class'][-1] == 'pick':
                            picks += [-1]
                    score_maps2 += [table.find('div', {'class': 'results-team-score'}).text]
            self.df['maps_url'][self.MATCH_ID] = arr
            self.df['maps_name'][self.MATCH_ID] = map_name
            self.df['score1_maps'][self.MATCH_ID] = score_maps1
            self.df['score2_maps'][self.MATCH_ID] = score_maps2
            if not picks:
                self.df['picks'][self.MATCH_ID] = "None"
            else:
                self.df['picks'][self.MATCH_ID] = picks
        except Exception as e:
            print(e)
            self.df['maps_url'][self.MATCH_ID] = "None"
            self.df['maps_name'][self.MATCH_ID] = "None"
            self.df['score1_maps'][self.MATCH_ID] = "None"
            self.df['score2_maps'][self.MATCH_ID] = "None"
            self.df['picks'][self.MATCH_ID] = "None"
    
    def add_maps(self):
        if 'maps_url' not in self.df:
            self.df['maps_url'] = ""
        if 'maps_name' not in self.df:
            self.df['maps_name'] = ""
        if 'score1_maps' not in self.df:
            self.df['score1_maps'] = ""
        if 'score2_maps' not in self.df:
            self.df['score2_maps'] = ""
        if 'picks' not in self.df:
            self.df['picks'] = ""
        if self.df['maps_url'][self.MATCH_ID] == "" or self.df['maps_name'][self.MATCH_ID] == "" or\
                self.df['score1_maps'][self.MATCH_ID] == "" or self.df['score2_maps'][self.MATCH_ID] == "" or\
                self.df['picks'][self.MATCH_ID] == "":
            self._get_maps_()
    
    def _get_url_teams_(self):
        self.parse_page()
        tmp = self.MATCH_PAGE.find('div', {'class': 'teamsBox'}).\
                find_all('div', {'class': 'team'})
        self.df['url_team1'][self.MATCH_ID] = tmp[0].find('a')['href']
        self.df['url_team2'][self.MATCH_ID] = tmp[1].find('a')['href']
    
    def add_url_teams(self):
        if 'url_team1' not in self.df:
            self.df['url_team1'] = ""
        if 'url_team2' not in self.df:
            self.df['url_team2'] = ""
        if self.df['url_team1'][self.MATCH_ID] == "" or self.df['url_team2'][self.MATCH_ID] == "":
            self._get_url_teams_()


MatchPageParams(df).add_all_params()


In [ ]:
df

### Descriptions for all columns
- 'match_url', - ссылка на матч
- 'event_url', - ссылка на турнир
- 'players_url_1', - ссылки на игроков 1ой команды
- 'players_url_2', - ссылки на игроков 2ой команды
- 'maps_url', - ссылки на статистики сыгранных карты
- 'maps_name' - названия карт
- 'score1_maps' - счет на каждой карте команды 1
- 'score2_maps' - счет на каждой карте команды 2
- 'picks' - пики команд; 1 - первая команды, -1 - вторая команда; если массив None, то карт < 2
- 'date', - дата матча
- 'total_maps', - всего планировалось сыграть карт (обычно 1, 3 или 5)
- 'maps_played', - сколько карт сыгралось в итоге
- 'score1', - счет 1ой команды
- 'score2', - счет 2ой команды
- 'h2h_wins1', - история побед 1ой команды над 2ой
- 'h2h_wins2', - история побед 2ой команды над 1ой
- 'rank1', - ранг 1ой команды
- 'rank2', - ранг 2ой команды

In [501]:
df.to_csv('data_common.csv')

# Add 10 last played maps

In [505]:
df = pd.read_csv('data_common.csv').set_index('Unnamed: 0')
# df

In [ ]:
TEAM_STAT = 'http://www.hltv.org/stats/teams/matches'
months = 3
last_maps = 20
TIMESLEEP = 0.01


class LastMaps:
    def __init__(self, df, start_index=0, finish_index=None):
        self.df = df
        self.TEAM_PAGE = None
        self.MATCH_ID = None
        self.start_index = start_index
        self.finish_index = finish_index
        self.start_date = None
        self.finish_date = None

    def add_all_params(self):
        time_start = time.time()
        for date, t1, t2, index in zip(df['date'][self.start_index:self.finish_index],
                                       df['url_team1'][self.start_index:self.finish_index],
                                       df['url_team2'][self.start_index:self.finish_index],
                                       range(len(df['date'][self.start_index:self.finish_index]))):
            self.MATCH_ID = index + self.start_index
            print('match id -', self.MATCH_ID, '; time spent -', time.time() - time_start)
            self.start_date = (datetime.strptime(date, '%Y-%m-%d') - relativedelta(months=months)).strftime("%Y-%m-%d")
            self.finish_date = (datetime.strptime(date, '%Y-%m-%d') - relativedelta(days=1)).strftime("%Y-%m-%d")
            self.TEAM_PAGE = get_parsed_page(TEAM_STAT +
                                             t1.split('/team')[-1] +
                                             f'?startDate={self.start_date}&endDate={self.finish_date}',
                                             proxyDict, proxyAuth)
            self.add_last_scores_and_ranks('_team1')
            self.TEAM_PAGE = get_parsed_page(TEAM_STAT +
                                             t2.split('/team')[-1] +
                                             f'?startDate={self.start_date}&endDate={self.finish_date}',
                                             proxyDict, proxyAuth)
            self.add_last_scores_and_ranks('_team2')
            


    def _get_last_score_(self, team_id, match_id):
        try:
            tmp = self.TEAM_PAGE.find('table', {'class': 'stats-table'}).\
                    find('tbody').find_all('tr')[match_id].find('span', {'class': 'statsDetail'}).text.split(' - ')
            self.df[f'last_maps{match_id + 1}_score{team_id}'][self.MATCH_ID] = tmp[0]
            self.df[f'last_maps{match_id + 1}_opponent_score{team_id}'][self.MATCH_ID] = tmp[1]
        except:
            self.df[f'last_maps{match_id + 1}_score{team_id}'][self.MATCH_ID] = 0
            self.df[f'last_maps{match_id + 1}_opponent_score{team_id}'][self.MATCH_ID] = 0

    def add_last_scores_and_ranks(self, team_id):
        for match_id in range(last_maps):
            if f'last_maps{match_id + 1}_score{team_id}' not in self.df:
                self.df[f'last_maps{match_id + 1}_score{team_id}'] = ""
            if f'last_maps{match_id + 1}_opponent_score{team_id}' not in self.df:
                self.df[f'last_maps{match_id + 1}_opponent_score{team_id}'] = ""
            if self.df[f'last_maps{match_id + 1}_score{team_id}'][self.MATCH_ID] == "" or\
                self.df[f'last_maps{match_id + 1}_opponent_score{team_id}'][self.MATCH_ID] == "":
                self._get_last_score_(team_id, match_id)



LastMaps(df).add_all_params()


In [ ]:
df

### Descriptions for new columns
- last_maps{id1}_score{team_id} - счет команды; id1 - номер карты, team_id - номер команды (team1 или team2)
- last_maps{id1}_opponent_score{team_id} - счет оппонента; id1 - номер карты, team_id - номер команды (team1 или team2)

In [510]:
df.to_csv('data_last_maps.csv')

# Add event prizepool and teams

In [511]:
df = pd.read_csv('data_last_maps.csv').set_index('Unnamed: 0')
# df

In [ ]:
TIMESLEEP = random.uniform(0.25, 0.35)

class PrizePool:
    global TIMESLEEP
    def __init__(self, df, start_index=0, finish_index=None):
        self.df = df
        self.EVENT_PAGE = None
        self.MATCH_ID = None
        self.start_index = start_index
        self.finish_index = finish_index
        self.event = None
    
    def add_all_params(self):
        time_start = time.time()
        for event, index in zip(self.df['event_url'][self.start_index:self.finish_index],
                         range(len(df['event_url'][self.start_index:self.finish_index]))):
            self.MATCH_ID = index + self.start_index
            print(PREFIX + event)
            print('match id -', self.MATCH_ID, '; time spent -', time.time() - time_start)
            self.event = event
            self.add_event()

    def _get_event_(self):
        self.EVENT_PAGE = get_parsed_page(PREFIX + self.event, proxyDict, proxyAuth)
        tmp = self.EVENT_PAGE.find('table', {'class': 'info'}).find('tbody')
        self.df['prize_pool'][self.MATCH_ID] = tmp.find('td', {'class': 'prizepool'}).text
        self.df['event_teams'][self.MATCH_ID] = tmp.find('td', {'class': 'teamsNumber'}).text
        self.df['event_type'][self.MATCH_ID] = tmp.find('td', {'class': 'location'}).\
                find('span', {'class': 'text-ellipsis'}).text
        
    
    def add_event(self):
        if 'prize_pool' not in self.df:
            self.df['prize_pool'] = ""
        if 'event_teams' not in self.df:
            self.df['event_teams'] = ""
        if 'event_type' not in self.df:
            self.df['event_type'] = ""
        if self.df['event_type'][self.MATCH_ID] == "" or\
            self.df['event_teams'][self.MATCH_ID] == "" or\
            self.df['prize_pool'][self.MATCH_ID] == "":
            self._get_event_()

            
PrizePool(df).add_all_params()
TIMESLEEP = 0.01

In [ ]:
def processing_parizepool():
    for i in range(len(df['prize_pool'])):
        if df['prize_pool'][i].split(' ')[0][0] == '$':
            df['prize_pool'][i] = re.sub(r'[^\d.]', '', df['prize_pool'][i].split(' ')[0])
        else:
            df['prize_pool'][i] = 0

def processing_eventteams():
    for i in range(len(df)):
        if df['event_teams'][i] != '':
            df['event_teams'][i] = re.sub(r'[^\d]', '', df['event_teams'][i])
        else:
            df['event_teams'][i] = 0

def processing_eventtype():
    for i in range(len(df)):
        if re.findall(r'(Online)', df['event_type'][i]):
            df['event_type'][i] = 'Online'
        else:
            df['event_type'][i] = 'Lan'

processing_parizepool()
processing_eventteams()
processing_eventtype()

### Descriptions for new columns
- event_type - тип турнира (Lan или Online)
- event_teams - количество команд на турнире
- prize_pool - призовой фонд турнира

In [516]:
df.to_csv('data_tour.csv')

# Add player stat

### Add playing time in teams

In [ ]:
df = pd.read_csv('data_tour.csv').set_index('Unnamed: 0')
df

In [ ]:
PL_STAT = 'https://www.hltv.org'
import random
TIMESLEEP = random.uniform(0.25, 0.4)


class PlStatInTeam:
    def __init__(self, df, start_index=0, finish_index=None):
        self.df = df
        self.PL_PAGE = None
        self.MATCH_ID = None
        self.start_index = start_index
        self.finish_index = finish_index
        self.period = None

    def add_all_params(self):
        time_start = time.time()
        for date, index in zip(df['date'][self.start_index:self.finish_index],
                               range(len(df.iloc[self.start_index:self.finish_index]))):
            self.MATCH_ID = index + self.start_index
            print('match id -', self.MATCH_ID, '; time spent -', time.time() - time_start)
            self.period = (datetime.now() - datetime.strptime(date, '%Y-%m-%d')).days
            self.add_5_players(1)
            self.add_5_players(2)
    
    def _get_player_stat_(self, team_id, player):
        print(team_id, player)
        player_url = (eval(df[f'players_url_{team_id}'][self.MATCH_ID])[player])
        if player_url:
            self.PL_PAGE = get_parsed_page(PL_STAT + player_url + '#tab-teamsBox')  #, proxyDict, proxyAuth)
            tmp = [i.find('div', {'class': 'stat'}).text for i in
                   self.PL_PAGE.find('div', {'class': 'tab-content', 'id': 'teamsBox'}).\
                   find_all('div', {'class': 'highlighted-stat'})]
            tmp = [0 if i == '-' else int(i) for i in tmp]
            self.df[f'player{player + 1}_teams_all_team{team_id}'][self.MATCH_ID] = tmp[0]
            self.df[f'player{player + 1}_days_in_current_team{team_id}'][self.MATCH_ID] = max(tmp[1] - self.period, 0)
            self.df[f'player{player + 1}_days_in_all_team{team_id}'][self.MATCH_ID] = max(tmp[2] - self.period, 0)
        else:
            self.df[f'player{player + 1}_teams_all_team{team_id}'][self.MATCH_ID] = 0
            self.df[f'player{player + 1}_days_in_current_team{team_id}'][self.MATCH_ID] = 0
            self.df[f'player{player + 1}_days_in_all_team{team_id}'][self.MATCH_ID] = 0

    def add_5_players(self, team_id):
        for player in range(5):
            if f'player{player + 1}_teams_all_team{team_id}' not in self.df:
                self.df[f'player{player + 1}_teams_all_team{team_id}'] = ""
            if f'player{player + 1}_days_in_current_team{team_id}' not in self.df:
                self.df[f'player{player + 1}_days_in_current_team{team_id}'] = ""
            if f'player{player + 1}_days_in_all_team{team_id}' not in self.df:
                self.df[f'player{player + 1}_days_in_all_team{team_id}'] = ""
            if self.df[f'player{player + 1}_days_in_current_team{team_id}'][self.MATCH_ID] == "" or\
                self.df[f'player{player + 1}_days_in_all_team{team_id}'][self.MATCH_ID] == "" or\
                self.df[f'player{player + 1}_teams_all_team{team_id}'][self.MATCH_ID] == "":
                self._get_player_stat_(team_id, player)
            

PlStatInTeam(df).add_all_params()
#eval(df['players_url_1'][0])
TIMESLEEP = 0.01

### Descriptions for new columns
playerID - номер игрока (от 1 до 5); team_id - номер команды (1 или 2)
- 'player{playerID}_days_in_current_team{team_id}' - дней, сколько игрок состоит в одной команде (0 или более)
- 'player{playerID}_days_in_all_team{team_id}' - дней, сколько всего игрок состоит в командах (0 или более)
- 'player{playerID}_teams_all_team{team_id}' - количество команд, в которых состоял игрок (0 или более)

In [ ]:
df.to_csv(f'data_stat_pl1.csv')
df

### Add full stat players

In [ ]:
df = pd.read_csv('data_stat_pl1.csv').set_index('Unnamed: 0')
df

In [ ]:
TIMESLEEP = random.uniform(0.25, 0.35)
PL_STAT_ALL = 'https://www.hltv.org/stats/players'
months = 3


class PlStatAll:
    def __init__(self, df, start_index=0, finish_index=None):
        self.df = df
        self.PL_PAGE = None
        self.MATCH_ID = None
        self.start_index = start_index
        self.finish_index = finish_index
        self.start_date = None
        self.finish_date = None

    def add_all_params(self):
        time_start = time.time()
        for date, index in zip(df['date'][self.start_index:self.finish_index],
                               range(len(df.iloc[self.start_index:self.finish_index]))):
            self.MATCH_ID = index + self.start_index
            print('match id -', self.MATCH_ID, '; time spent -', time.time() - time_start)
            self.start_date = (datetime.strptime(date, '%Y-%m-%d') - relativedelta(months=months)).strftime("%Y-%m-%d")
            self.finish_date = (datetime.strptime(date, '%Y-%m-%d') - relativedelta(days=1)).strftime("%Y-%m-%d")
            self.add_5_players_stat(1)
            self.add_5_players_stat(2)



    def _get_player_stat_(self, team_id, player_id, params_1, params_2):
        def find_row1(stats_row, param):
            for i in stats_row:
                if i.find('span').text == param:
                    return i.find_all('span')[-1].text
            return 0
        
        def find_row2(col_custom, param):
            for i in col_custom:
                if i.find('div', {'class': 'rating-description'}).text == param:
                    tmp = [i.find('div', {'class': 'rating-value'}).text, i.find('div', {'class': 'rating-maps'}).text]
                    return [k if k != '-' else 0 for k in tmp]
            return 0, 0
        
        print(team_id, player_id)
        player_url = (eval(df[f'players_url_{team_id}'][self.MATCH_ID])[player_id])
        if player_url:
            self.PL_PAGE = get_parsed_page(PL_STAT_ALL + player_url.split('/player')[-1] +
                                           f'?startDate={self.start_date}&endDate={self.finish_date}',
                                           proxyDict, proxyAuth)
            # age
            self.df[f'age_player{player_id + 1}_team{team_id}'][self.MATCH_ID] = self.PL_PAGE.\
                    find('div', {'class': 'playerSummaryStatBox'}).\
                    find('div', {'class': 'summaryPlayerAge'}).text
            # stat params1
            stat1 = self.PL_PAGE.find('div', {'class': 'statistics'}).find_all('div', {'class': 'stats-row'})
            for param in params_1:
                self.df[f'{param}_player{player_id + 1}_team{team_id}'][self.MATCH_ID] = find_row1(stat1, param)
            # stat params2
            stat2 = self.PL_PAGE.find('div', {'class': 'featured-ratings-container'}).\
                    find_all('div', {'class': 'col-custom'})
            for param in params_2:
                (self.df[f'{param}_player{player_id + 1}_team{team_id}'][self.MATCH_ID],
                self.df[f'{param}_maps_player{player_id + 1}_team{team_id}'][self.MATCH_ID]) = \
                find_row2(stat2, param)
        else:
            for param in params_1:
                self.df[f'{param}_player{player_id + 1}_team{team_id}'][self.MATCH_ID] = 0
            for param in params_2:
                self.df[f'{param}_player{player_id + 1}_team{team_id}'][self.MATCH_ID] = 0
                self.df[f'{param}_maps_player{player_id + 1}_team{team_id}'][self.MATCH_ID] = 0
            self.df[f'age_player{player_id + 1}_team{team_id}'][self.MATCH_ID] = 0
            

    def add_5_players_stat(self, team_id):
        def check_null_in_line(player_id, params_1, params_2):
            for param in params_1:
                if self.df[f'{param}_player{player_id + 1}_team{team_id}'][self.MATCH_ID] == "":
                    return 1
            for param in params_2:
                if self.df[f'{param}_player{player_id + 1}_team{team_id}'][self.MATCH_ID] == "" or \
                    self.df[f'{param}_maps_player{player_id + 1}_team{team_id}'][self.MATCH_ID] == "":
                    return 1
            if self.df[f'age_player{player_id + 1}_team{team_id}'][self.MATCH_ID] == "":
                return 1
            return 0

        for player in range(5):
            params_1 = ['Total kills', 'Headshot %', 'Total deaths', 'K/D Ratio',
                        'Damage / Round', 'Grenade dmg / Round', 'Maps played', 'Rounds played',
                        'Kills / round', 'Assists / round', 'Deaths / round',
                        'Saved by teammate / round', 'Saved teammates / round', 'Rating 2.0']
            params_2 = ['vs top 5 opponents', 'vs top 10 opponents', 'vs top 20 opponents',
                        'vs top 30 opponents', 'vs top 50 opponents']
            for param in params_1:
                if f'{param}_player{player + 1}_team{team_id}' not in self.df:
                    self.df[f'{param}_player{player + 1}_team{team_id}'] = ""
            for param in params_2:
                if f'{param}_player{player + 1}_team{team_id}' not in self.df:
                    self.df[f'{param}_player{player + 1}_team{team_id}'] = ""
                if f'{param}_maps_player{player + 1}_team{team_id}' not in self.df:
                    self.df[f'{param}_maps_player{player + 1}_team{team_id}'] = ""
            if f'age_player{player + 1}_team{team_id}' not in self.df:
                self.df[f'age_player{player + 1}_team{team_id}'] = ""
            if check_null_in_line(player, params_1, params_2):
                self._get_player_stat_(team_id, player, params_1, params_2)



PlStatAll(df).add_all_params()
TIMESLEEP = 0.01

In [ ]:
params_2 = ['vs top 5 opponents', 'vs top 10 opponents', 'vs top 20 opponents',
            'vs top 30 opponents', 'vs top 50 opponents']
params_1 = ['Total kills', 'Headshot %', 'Total deaths', 'K/D Ratio',
            'Damage / Round', 'Grenade dmg / Round', 'Maps played', 'Rounds played',
            'Kills / round', 'Assists / round', 'Deaths / round',
            'Saved by teammate / round', 'Saved teammates / round', 'Rating 2.0']


def processing_age(df):
    for i in range(len(df)):
        for team_id in range(1, 3):
            for player_id in range(5):
                if df[f'age_player{player_id + 1}_team{team_id}'][i] == '':
                    df[f'age_player{player_id + 1}_team{team_id}'][i] = 0
                df[f'age_player{player_id + 1}_team{team_id}'][i] =\
                        re.sub(r'[^\d]', '', str(df[f'age_player{player_id + 1}_team{team_id}'][i]))


def processing_maps(df):
    for i in range(len(df)):
        for team_id in range(1, 3):
            for player_id in range(5):
                for param in params_2:
                    if str(df[f'{param}_maps_player{player_id + 1}_team{team_id}'][i])[0] == '(':
                        df[f'{param}_maps_player{player_id + 1}_team{team_id}'][i] =\
                                df[f'{param}_maps_player{player_id + 1}_team{team_id}'][i].\
                                split('(')[1].split(' maps')[0]


def processing_procent(df):
    for i in range(len(df)):
        for team_id in range(1, 3):
            for player_id in range(5):
                for param in params_1:  # Headshot %_player1_team1
                    if str(df[f'{param}_player{player_id + 1}_team{team_id}'][i])[-1] == '%':
                        df[f'{param}_player{player_id + 1}_team{team_id}'][i] =\
                            float(str(df[f'{param}_player{player_id + 1}_team{team_id}'][i])[:-1]) / 100

processing_age(df)
processing_maps(df)
processing_procent(df)
df

### Descriptions for new columns
- '{param}_player{player}_team{team_id}' - статистика игрока
- '{param}_maps_player{player_id}_team{team_id}' - сыграно карт для рассчета статистики (только некоторых параметров)
- 'age_player{player_id}_team{team_id}' - возраст игрока

In [577]:
# save
df.to_csv(f'data_stat_pl2.csv')

# Add all map stat

In [22]:
df = pd.read_csv('data_stat_pl2_test.csv').set_index('Unnamed: 0')
df

,match_url,url_team1,url_team2,event_url,players_url_1,players_url_2,total_maps,maps_played,maps_url,maps_name,...,5last_match2_opponent_score2,5last_match3_total_maps2,5last_match3_score2,5last_match3_opponent_score2,5last_match4_total_maps2,5last_match4_score2,5last_match4_opponent_score2,5last_match5_total_maps2,5last_match5_score2,5last_match5_opponent_score2
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,/matches/2341924/hard-legion-vs-hellraisers-lo...,/team/10421/hard-legion,/team/5310/hellraisers,/events/5359/lootbet-season-7,"['/player/12027/Krad', '/player/13982/Forester...","['/player/11250/Flarich', '/player/8786/jR', '...",3,3,['/stats/matches/mapstatsid/103958/hard-legion...,"['Inferno', 'Dust2', 'Train']",...,0,Dust2,1,0,Best of 3,0,1,Best of 3,0,1
1,/matches/2341888/bantz-vs-truckers-with-attitu...,/team/10712/bantz,/team/10713/truckers-with-attitude,/events/5319/lpl-pro-league-season-5,"['/player/18737/Falcon', '/player/20017/motion...","['/player/18736/pz', '/player/11806/JD', '/pla...",3,3,['/stats/matches/mapstatsid/103959/bantz-vs-tr...,"['Vertigo', 'Inferno', 'Mirage']",...,0,Best of 3,1,0,Mirage,0,1,Train,1,0
2,/matches/2341576/natus-vincere-vs-mad-lions-dr...,/team/4608/natus-vincere,/team/8362/mad-lions,/events/5285/dreamhack-masters-spring-2020-europe,"['/player/7594/flamie', '/player/7998/s1mple',...","['/player/10330/acoR', '/player/14148/sjuush',...",3,2,['/stats/matches/mapstatsid/103961/natus-vince...,"['Mirage', 'Overpass']",...,0,Best of 3,0,1,Train,0,1,Train,1,0
3,/matches/2342056/pc419-vs-overt-esea-mdl-seaso...,/team/9750/pc419,/team/10711/overt,/events/5264/esea-mdl-season-34-australia,"['/player/11793/dash', '/player/11803/Fluent%2...","['/player/12584/vax', '/player/11808/xeL', '/p...",1,1,['/stats/matches/mapstatsid/103960/overt-vs-pc...,['Overpass'],...,1,Train,0,1,Inferno,0,1,Train,1,0
4,/matches/2342055/order-vs-skyfire-esea-mdl-sea...,/team/8668/order,/team/6161/skyfire,/events/5264/esea-mdl-season-34-australia,"['/player/11139/aliStair', '/player/16323/Vali...","['/player/11492/Noisia', '/player/8770/MoeycQ'...",1,1,['/stats/matches/mapstatsid/103957/skyfire-vs-...,['Dust2'],...,1,Nuke,1,0,Nuke,0,1,Nuke,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,/matches/2337816/giants-vs-ftw-master-league-p...,/team/4949/giants,/team/7379/ftw,/events/5028/master-league-portugal-iv,"['/player/629/fox', '/player/630/MUTiRiS', '/p...","['/player/16844/psh', '/player/16843/fakeS2', ...",1,1,['/stats/matches/mapstatsid/95171/giants-vs-ftw'],['Train'],...,1,Best of 3,0,1,Inferno,0,1,Train,0,1
3996,/matches/2337883/winstrike-vs-forze-weplay-for...,/team/9183/winstrike,/team/8135/forze,/events/5013/weplay-forge-of-masters-season-2-...,"['/player/2131/bondik', '/player/13281/El1an',...","['/player/8726/facecrack', '/player/13157/Jerr...",3,3,['/stats/matches/mapstatsid/95157/winstrike-vs...,"['Nuke', 'Train', 'Dust2']",...,0,Best of 3,1,0,Best of 3,1,0,Best of 3,1,0
3997,/matches/2337888/mvp-pk-vs-absolute-extremesla...,/team/7354/mvp-pk,/team/7581/absolute,/events/4681/extremesland-2019-asia-finals,"['/player/205/glow', '/player/11188/xeta', '/p...","['/player/11430/crow', '/player/9585/Laz', '/p...",3,3,['/stats/matches/mapstatsid/95156/absolute-vs-...,"['Inferno', 'Dust2', 'Mirage']",...,0,Mirage,0,1,Best of 3,0,1,Best of 2,1,0


In [23]:
id_maps = {
'Dust2': 31,
'Inferno': 33,
'Mirage': 32,
'Nuke': 34,
'Overpass': 40,
'Train': 35,
'Vertigo': 46,
}

In [ ]:
TIMESLEEP = random.uniform(0.2, 0.35)
last_played_maps = 10
months = 3
# map_stat_url for a team: http://www.hltv.org/stats/teams/map/35/4991/fnatic
MAP_STAT_PR = 'https://www.hltv.org/stats/teams/map/'


class AllMapsStat:
    def __init__(self, df, start_index=0, finish_index=None):
        self.df = df
        self.STAT_PAGE = None
        self.idx = None
        self.start_index = start_index
        self.finish_index = finish_index
        self.start_date = None
        self.finish_date = None

    def add_all_params(self):
        time_start = time.time()
        for date, index in zip(self.df['date'][self.start_index:self.finish_index],
                       range(len(self.df.iloc[self.start_index:self.finish_index]))):
            #print(index)
            self.idx = index + self.start_index
            print('match id -', self.idx, '; time spent -', time.time() - time_start)
            #print(maps_name)
            #print(df['url_team1'][self.MATCH_ID])
            self.start_date = (datetime.strptime(date, '%Y-%m-%d') - relativedelta(months=months)).strftime("%Y-%m-%d")
            self.finish_date = (datetime.strptime(date, '%Y-%m-%d') - relativedelta(days=1)).strftime("%Y-%m-%d")
            self.add_map_in_df([i for i in id_maps])
    
    
    
    def _get_stat_(self, team_id, params, params1, params2, map_name):
        def _get_params_(team_id, param, table):
            for i in table:
                if i.find('span').text == param:
                    return i.find_all('span')[-1].text
            print('Not find')
            return 0

        def _get_params1_(team_id, param, table):
            for i in table:
                if i.find_all('div')[1].text == param:
                    return i.find('div').text
            print('Not find')
            return 0
    
        def _get_params2_(team_id, param, table):
            score = table.find('td', {'class': 'statsTeamMapResult'}).text
            score = score.split(' - ')
            return score
            
        tmp = self.STAT_PAGE.find_all('div', {'class': 'columns'})
        table = tmp[0].find('div', {'class': 'stats-rows'}).\
                find_all('div', {'class': 'stats-row'})
        for param in params:
            self.df[param + map_name + f'_team{team_id}'][self.idx] = _get_params_(team_id, param, table)
        table = tmp[1].find_all('div', {'class': 'big-padding'})
        for param in params1:
            self.df[param + map_name + f'_team{team_id}'][self.idx] = _get_params1_(team_id, param, table)
        table = self.STAT_PAGE.find('table', {'class': 'stats-table'}).\
                find('tbody').find_all('tr')
        for pos in range(len(params2)):
            param = params2[pos]
            if pos < len(table):
                [self.df[param + map_name + f'_team{team_id}'][self.idx],
                 self.df[param + map_name + '_opponent' + f'_team{team_id}'][self.idx]] = _get_params2_(team_id, param, table[pos])
            else:
                self.df[param + map_name + f'_team{team_id}'][self.idx] = 0
                self.df[param + map_name + '_opponent' + f'_team{team_id}'][self.idx] = 0

    
    def add_map_in_df(self, maps_name):
        def check_null_in_line(team_id, params, params1, params2):
            for param in params:
                if self.df[param + map_name + f'_team{team_id}'][self.idx] == "":
                    return 1
            for param in params1:
                if self.df[param + map_name + f'_team{team_id}'][self.idx] == "":
                    return 1
            for param in params2:
                if self.df[param + map_name + f'_team{team_id}'][self.idx] == "" or\
                        self.df[param + map_name + '_opponent' + f'_team{team_id}'][self.idx] == "":
                    return 1
            return 0
        
        for map_name in maps_name:
            # print(map_name)
            if map_name:
                columns = ['Times played', 'Total rounds played', 'Rounds won', 'Win percent',
                           'Pistol rounds', 'Pistol rounds won', 'Pistol round win percent',
                           'CT round win percent', 'T round win percent']
                columns1 = ['Round win percent after getting first kill',
                            'Round win percent after receiving first death']
                columns2 = [f'map_played{k + 1}'
                            for k in range(last_played_maps)]
                for team_id in range(1, 3):
                    for i in columns:
                        if i + map_name + f'_team{team_id}' not in df:
                            self.df[i + map_name + f'_team{team_id}'] = ""
                    for i in columns1:
                        if i + map_name + f'_team{team_id}' not in df:
                            self.df[i + map_name + f'_team{team_id}'] = ""
                    for i in columns2:
                        if i + map_name + f'_team{team_id}' not in df:
                            self.df[i + map_name + f'_team{team_id}'] = ""
                            self.df[i + map_name + '_opponent' + f'_team{team_id}'] = ""
                # print(self.idx)
                for team_id in range(1, 3):
                    if check_null_in_line(team_id, columns, columns1, columns2):
                        url = MAP_STAT_PR + str(id_maps[map_name]) + '/' +\
                              (df[f'url_team{team_id}'][self.idx]).split('/team/')[-1] +\
                              f'?startDate={self.start_date}&endDate={self.finish_date}'
                        print(url)
                        self.STAT_PAGE = get_parsed_page(url, proxyDict, proxyAuth)
                        self._get_stat_(team_id, columns, columns1, columns2, map_name)

cl =  AllMapsStat(df)
cl.add_all_params()

In [ ]:
columns = ['Times played', 'Total rounds played', 'Rounds won', 'Win percent',
                           'Pistol rounds', 'Pistol rounds won', 'Pistol round win percent',
                           'CT round win percent', 'T round win percent']
columns1 = ['Round win percent after getting first kill',
                            'Round win percent after receiving first death']
for i in columns:
    print(list(df.filter(regex=i, axis=1).columns))
    for k in list(df.filter(regex=i, axis=1).columns):
        for idx in range(len(df)):
            if str(df[k][idx])[-1] == '%':
                df[k][idx] = float(str(df[k][idx])[:-1]) / 100
for i in columns1:
    print(list(df.filter(regex=i, axis=1).columns))
    for k in list(df.filter(regex=i, axis=1).columns):
        for idx in range(len(df)):
            if str(df[k][idx])[-1] == '%':
                df[k][idx] = float(str(df[k][idx])[:-1]) / 100

In [16]:
df

,match_url,url_team1,url_team2,event_url,players_url_1,players_url_2,total_maps,maps_played,maps_url,maps_name,...,map_played6Vertigo_team2,map_played6Vertigo_opponent_team2,map_played7Vertigo_team2,map_played7Vertigo_opponent_team2,map_played8Vertigo_team2,map_played8Vertigo_opponent_team2,map_played9Vertigo_team2,map_played9Vertigo_opponent_team2,map_played10Vertigo_team2,map_played10Vertigo_opponent_team2
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,/matches/2341924/hard-legion-vs-hellraisers-lo...,/team/10421/hard-legion,/team/5310/hellraisers,/events/5359/lootbet-season-7,"['/player/12027/Krad', '/player/13982/Forester...","['/player/11250/Flarich', '/player/8786/jR', '...",3,3,['/stats/matches/mapstatsid/103958/hard-legion...,"['Inferno', 'Dust2', 'Train']",...,16,10,0,0,0,0,0,0,0,0
1,/matches/2341888/bantz-vs-truckers-with-attitu...,/team/10712/bantz,/team/10713/truckers-with-attitude,/events/5319/lpl-pro-league-season-5,"['/player/18737/Falcon', '/player/20017/motion...","['/player/18736/pz', '/player/11806/JD', '/pla...",3,3,['/stats/matches/mapstatsid/103959/bantz-vs-tr...,"['Vertigo', 'Inferno', 'Mirage']",...,0,0,0,0,0,0,0,0,0,0
2,/matches/2341576/natus-vincere-vs-mad-lions-dr...,/team/4608/natus-vincere,/team/8362/mad-lions,/events/5285/dreamhack-masters-spring-2020-europe,"['/player/7594/flamie', '/player/7998/s1mple',...","['/player/10330/acoR', '/player/14148/sjuush',...",3,2,['/stats/matches/mapstatsid/103961/natus-vince...,"['Mirage', 'Overpass']",...,0,0,0,0,0,0,0,0,0,0
3,/matches/2342056/pc419-vs-overt-esea-mdl-seaso...,/team/9750/pc419,/team/10711/overt,/events/5264/esea-mdl-season-34-australia,"['/player/11793/dash', '/player/11803/Fluent%2...","['/player/12584/vax', '/player/11808/xeL', '/p...",1,1,['/stats/matches/mapstatsid/103960/overt-vs-pc...,['Overpass'],...,0,0,0,0,0,0,0,0,0,0
4,/matches/2342055/order-vs-skyfire-esea-mdl-sea...,/team/8668/order,/team/6161/skyfire,/events/5264/esea-mdl-season-34-australia,"['/player/11139/aliStair', '/player/16323/Vali...","['/player/11492/Noisia', '/player/8770/MoeycQ'...",1,1,['/stats/matches/mapstatsid/103957/skyfire-vs-...,['Dust2'],...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,/matches/2337816/giants-vs-ftw-master-league-p...,/team/4949/giants,/team/7379/ftw,/events/5028/master-league-portugal-iv,"['/player/629/fox', '/player/630/MUTiRiS', '/p...","['/player/16844/psh', '/player/16843/fakeS2', ...",1,1,['/stats/matches/mapstatsid/95171/giants-vs-ftw'],['Train'],...,0,0,0,0,0,0,0,0,0,0
3996,/matches/2337883/winstrike-vs-forze-weplay-for...,/team/9183/winstrike,/team/8135/forze,/events/5013/weplay-forge-of-masters-season-2-...,"['/player/2131/bondik', '/player/13281/El1an',...","['/player/8726/facecrack', '/player/13157/Jerr...",3,3,['/stats/matches/mapstatsid/95157/winstrike-vs...,"['Nuke', 'Train', 'Dust2']",...,0,0,0,0,0,0,0,0,0,0
3997,/matches/2337888/mvp-pk-vs-absolute-extremesla...,/team/7354/mvp-pk,/team/7581/absolute,/events/4681/extremesland-2019-asia-finals,"['/player/205/glow', '/player/11188/xeta', '/p...","['/player/11430/crow', '/player/9585/Laz', '/p...",3,3,['/stats/matches/mapstatsid/95156/absolute-vs-...,"['Inferno', 'Dust2', 'Mirage']",...,0,0,0,0,0,0,0,0,0,0


In [17]:
# save
df.to_csv(f'data_stat_all_maps.csv')

# Add each map in dataframe

In [ ]:
df = pd.read_csv('data_stat_pl2.csv').set_index('Unnamed: 0')
df

In [ ]:
df.iloc[:, 9:13]

In [463]:
id_maps = {
'Dust2': 31,
'Inferno': 33,
'Mirage': 32,
'Nuke': 34,
'Overpass': 40,
'Train': 35,
'Vertigo': 46,
'Cache': 29,
'Cobblestone': 39,
}

In [ ]:
TIMESLEEP = random.uniform(0.2, 0.35)
last_played_maps = 10
months = 3
# map_stat_url for a team: http://www.hltv.org/stats/teams/map/35/4991/fnatic
MAP_STAT_PR = 'https://www.hltv.org/stats/teams/map/'


class MapsStatTeam:
    def __init__(self, df, start_index=0, finish_index=None):
        self.df = df
        self.dfNew = None
        self.STAT_PAGE = None
        self.MATCH_ID = None
        self.idx = 0
        self.start_index = start_index
        self.finish_index = finish_index
        self.start_date = None
        self.finish_date = None

    def add_all_params(self):
        time_start = time.time()
        for maps_url, maps_name, date, score1_maps, score2_maps, picks, index\
            in zip(self.df['maps_url'][self.start_index:self.finish_index],
                       self.df['maps_name'][self.start_index:self.finish_index],
                       self.df['date'][self.start_index:self.finish_index],
                       self.df['score1_maps'][self.start_index:self.finish_index],
                       self.df['score2_maps'][self.start_index:self.finish_index],
                       self.df['picks'][self.start_index:self.finish_index],
                       range(len(self.df.iloc[self.start_index:self.finish_index]))):
            #print(index)
            self.MATCH_ID = index + self.start_index
            print('match id -', self.MATCH_ID, '; time spent -', time.time() - time_start)
            #print(maps_name)
            #print(df['url_team1'][self.MATCH_ID])
            self.start_date = (datetime.strptime(date, '%Y-%m-%d') - relativedelta(months=months)).strftime("%Y-%m-%d")
            self.finish_date = (datetime.strptime(date, '%Y-%m-%d') - relativedelta(days=1)).strftime("%Y-%m-%d")
            if eval(maps_name):
                self.add_map_in_df(eval(maps_url), eval(maps_name),
                                   eval(score1_maps), eval(score2_maps), eval(picks))
    
    
    
    def _get_stat_(self, team_id, params, params1, params2, score1_map, score2_map, counter, picks):
        def _get_params_(team_id, param, table):
            for i in table:
                if i.find('span').text == param:
                    return i.find_all('span')[-1].text
            print('Not find')
            return 0

        def _get_params1_(team_id, param, table):
            for i in table:
                if i.find_all('div')[1].text == param:
                    return i.find('div').text
            print('Not find')
            return 0
    
        def _get_params2_(team_id, param, table):
            score = table.find('td', {'class': 'statsTeamMapResult'}).text
            score = score.split(' - ')
            return score
            
        tmp = self.STAT_PAGE.find_all('div', {'class': 'columns'})
        table = tmp[0].find('div', {'class': 'stats-rows'}).\
                find_all('div', {'class': 'stats-row'})
        for param in params:
            self.dfNew[param + f'_team{team_id}'][self.idx] = _get_params_(team_id, param, table)
        table = tmp[1].find_all('div', {'class': 'big-padding'})
        for param in params1:
            self.dfNew[param + f'_team{team_id}'][self.idx] = _get_params1_(team_id, param, table)
        table = self.STAT_PAGE.find('table', {'class': 'stats-table'}).\
                find('tbody').find_all('tr')
        for pos in range(len(params2)):
            param = params2[pos]
            if pos < len(table):
                [self.dfNew[param + f'_team{team_id}'][self.idx],
                self.dfNew[param + '_opponent' + f'_team{team_id}'][self.idx]] = _get_params2_(team_id, param, table[pos])
            else:
                self.dfNew[param + f'_team{team_id}'][self.idx] = 0
                self.dfNew[param + '_opponent' + f'_team{team_id}'][self.idx] = 0
        self.dfNew['map_score1'][self.idx] = score1_map
        self.dfNew['map_score2'][self.idx] = score2_map
        if picks and counter < len(picks):
            if picks[counter] == 1:
                self.dfNew['pick1'][self.idx] = 1
                self.dfNew['pick2'][self.idx] = 0
            elif picks[counter] == -1:
                self.dfNew['pick2'][self.idx] = 1
                self.dfNew['pick1'][self.idx] = 0
        else:
            self.dfNew['pick1'][self.idx] = 0
            self.dfNew['pick2'][self.idx] = 0

    
    def add_map_in_df(self, maps_url, maps_name, score1_maps, score2_maps, picks):
        def check_null_in_line(team_id, params, params1, params2):
            for param in params:
                if self.dfNew[param + f'_team{team_id}'][self.idx] == "":
                    return 1
            for param in params1:
                if self.dfNew[param + f'_team{team_id}'][self.idx] == "":
                    return 1
            for param in params2:
                if self.dfNew[param + f'_team{team_id}'][self.idx] == "" or\
                        self.dfNew[param + '_opponent' + f'_team{team_id}'][self.idx] == "":
                    return 1
            for param in ['map_score1', 'map_score2', 'pick1', 'pick2']:
                if self.dfNew[param][self.idx] == "":
                    return 1
            return 0
        
        for map_url, map_name, score1_map, score2_map, counter \
                in zip(maps_url, maps_name, score1_maps, score2_maps, range(len(maps_name))):
            # print(map_name)
            if map_name:
                columns = ['Times played', 'Total rounds played', 'Rounds won', 'Win percent',
                           'Pistol rounds', 'Pistol rounds won', 'Pistol round win percent',
                           'CT round win percent', 'T round win percent']
                columns1 = ['Round win percent after getting first kill',
                            'Round win percent after receiving first death']
                columns2 = [f'current_map_played_{k + 1}'
                            for k in range(last_played_maps)]
                if self.dfNew is None:
                    self.dfNew = self.df.iloc[self.MATCH_ID:self.MATCH_ID + 1].copy()
                    self.idx = 0
                    for team_id in range(1, 3):
                        for i in ['map_score1', 'map_score2', 'pick1', 'pick2']:
                            self.dfNew[i] = ""
                        for i in columns:
                            self.dfNew[i + f'_team{team_id}'] = ""
                        for i in columns1:
                            self.dfNew[i + f'_team{team_id}'] = ""
                        for i in columns2:
                            self.dfNew[i + f'_team{team_id}'] = ""
                            self.dfNew[i + '_opponent' + f'_team{team_id}'] = ""
                else:
                    self.dfNew = self.dfNew.append(self.df.iloc[self.MATCH_ID:self.MATCH_ID + 1].copy(), ignore_index=True)
                    self.idx += 1
                # print(self.idx)
                for team_id in range(1, 3):
                    #if check_null_in_line(team_id, columns, columns1, columns2):
                    url = MAP_STAT_PR + str(id_maps[map_name]) + '/' +\
                          (df[f'url_team{team_id}'][self.MATCH_ID]).split('/team/')[-1] +\
                          f'?startDate={self.start_date}&endDate={self.finish_date}'
                    print(url)
                    self.STAT_PAGE = get_parsed_page(url, proxyDict, proxyAuth)
                    self._get_stat_(team_id, columns, columns1, columns2,
                                    score1_map, score2_map, counter, picks)

cl = MapsStatTeam(df)
cl.add_all_params()
cl.dfNew

In [ ]:
cl.dfNew.iloc[:, 383:]

In [549]:
df = cl.dfNew

In [ ]:
columns = ['Times played', 'Total rounds played', 'Rounds won', 'Win percent',
           'Pistol rounds', 'Pistol rounds won', 'Pistol round win percent',
           'CT round win percent', 'T round win percent']
columns1 = ['Round win percent after getting first kill',
            'Round win percent after receiving first death']

def processing_params():
    def process(i, param, team_id):
        if str(df[f'{param}_team{team_id}'][i])[-1] == '%':
            df[f'{param}_team{team_id}'][i] =\
                float(str(df[f'{param}_team{team_id}'][i])[:-1]) / 100
 
    for i in range(len(df)):
        for team_id in range(1, 3):
            for param in columns:
                process(i, param, team_id)
            for param in columns1:
                process(i, param, team_id)


processing_params()

### Descriptions for new columns
- '{param}_team{team_id}' - статистика команды на конкретной карте
- 'current_map_played{№}_team{team_id}' - счет команды на конкретной картее из истории игр
- 'current_map_played{№}_opponent_team{team_id}' - счет оппонента на конкретной карте из истории игр
- 'map_score1' - счет 1ой команды на карте
- 'map_score2' - счет 2ой команды на карте
- 'pick1' - если равно 1 - пик 1ой команды; если 0 - не пик 1ой команды
- 'pick2' - если равно 1 - пик 2ой команды; если 0 - не пик 2ой команды

In [ ]:
# save
df.to_csv(f'data_each_map_months{months}.csv')
df

In [459]:
df = pd.read_csv('data_1_each_map.csv').set_index('Unnamed: 0')

In [461]:
np.unique(df['Win percent_team1'])

array([0.   , 0.111, 0.125, 0.133, 0.143, 0.154, 0.167, 0.182, 0.2  ,
       0.222, 0.231, 0.25 , 0.263, 0.267, 0.273, 0.286, 0.3  , 0.308,
       0.312, 0.333, 0.353, 0.357, 0.364, 0.375, 0.385, 0.389, 0.4  ,
       0.412, 0.417, 0.421, 0.429, 0.435, 0.438, 0.444, 0.45 , 0.455,
       0.462, 0.467, 0.471, 0.474, 0.476, 0.5  , 0.524, 0.526, 0.529,
       0.533, 0.538, 0.545, 0.55 , 0.556, 0.56 , 0.562, 0.565, 0.571,
       0.579, 0.583, 0.588, 0.591, 0.6  , 0.611, 0.615, 0.619, 0.625,
       0.632, 0.636, 0.643, 0.647, 0.65 , 0.652, 0.667, 0.679, 0.682,
       0.684, 0.688, 0.692, 0.7  , 0.706, 0.714, 0.722, 0.727, 0.733,
       0.737, 0.75 , 0.762, 0.765, 0.769, 0.773, 0.778, 0.786, 0.789,
       0.8  , 0.815, 0.818, 0.821, 0.824, 0.826, 0.828, 0.833, 0.846,
       0.857, 0.867, 0.875, 0.882, 0.889, 0.9  , 0.909, 0.929, 0.933,
       1.   ])

['Nuke']

In [421]:
for i, k, in zip(eval(df['maps_url'][52]), eval(df['maps_name'][52])):
    print(i, k)

/stats/matches/mapstatsid/103630/astralis-vs-ence Nuke


In [442]:
df['url_team2'][107]

'/team/6947/teamone'

In [431]:
df['maps_name'][52]

"['Nuke']"

In [ ]:
df = pd.read_csv('data_1_stat_all_players.csv').set_index('Unnamed: 0')
df

In [ ]:
TIMESLEEP = random.uniform(0.25, 0.35)


class MapsInMatch:
    def __init__(self, df, start_index=0, finish_index=None):
        self.df = df
        self.dfNew = None
        self.MAP_PAGE = None
        self.MATCH_ID = None
        self.idx = 0
        self.start_index = start_index
        self.finish_index = finish_index

    def add_all_params(self):
        time_start = time.time()
        for map_arr, index in zip(df['maps_url'][self.start_index:self.finish_index],
                               range(len(df.iloc[self.start_index:self.finish_index]))):
            self.MATCH_ID = index + self.start_index
            print('match id -', self.MATCH_ID, '; time spent -', time.time() - time_start)
            print(map_arr)
            if map_arr:
                self.add_map_in_df(eval(map_arr))
    
    def _get_map_name_(self):
        box = self.MAP_PAGE.find('div', {'class': 'match-info-box-con'}).\
                find_all('div', {'class': 'match-info-box'})
        for row in box:
            print(row)
        pass
    
    
    
    def add_map_in_df(self, map_arr):
        for map_url in map_arr:
            if map_url:
                columns = [ct_team1', 'ct_team2', 't_team1', 't_team2',
                           'first_kills_team1', 'first_kills_team2', 'clutches_team1',
                           'clutches_team2', 'round1', 'round15']
                if self.dfNew is None:
                    self.dfNew = self.df.iloc[self.MATCH_ID:self.MATCH_ID + 1].copy()
                    self.idx = 0
                    for i in columns:
                        self.dfNew[i] = ""
                else:
                    self.dfNew = self.dfNew.append(self.df.iloc[self.MATCH_ID:self.MATCH_ID + 1], ignore_index=True)
                    self.idx += 1
                self.MAP_PAGE = get_parsed_page(PREFIX + map_url, proxyDict, proxyAuth)
                self._get_map_name_()
            





def index_map_from_name(name):
    if name not in id_maps:
        return 'Not valid map'
    return id_maps[name]


def add_name_each_map_in_match(df):
    return df


def add_url_map_stat_team(df):
    return df


def add_score_each_side(df):
    return df


def transform_df(df):
    return df

def add_pistol_rounds(df):
    return df





MapsInMatch(df, 0, 1).add_all_params()

## Check and find bad columns (not possible to conert to float)

In [ ]:
start_index = 360

for i in range(start_index, len(df.iloc[1, :])):
    print(i)
    df.iloc[:, i].astype(int)


In [ ]:
df.append(df.iloc[0, :-1], ignore_index=True)